In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

Loading the dataset

In [2]:
data = pd.read_csv('../../datasets/medical_data.csv')
data.head()

,Patient_Problem,Disease,Prescription
0,"Constant fatigue and muscle weakness, struggli...",Chronic Fatigue Syndrome,"Cognitive behavioral therapy, graded exercise ..."
1,"Frequent severe migraines, sensitivity to ligh...",Migraine with Aura,"Prescription triptans, avoid triggers like bri..."
2,"Sudden weight gain and feeling cold, especiall...",Hypothyroidism,Levothyroxine to regulate thyroid hormone levels.
3,"High fever, sore throat, and swollen lymph nod...",Mononucleosis,"Rest and hydration, ibuprofen for pain."
4,"Excessive thirst and frequent urination, dry m...",Diabetes Mellitus,Insulin therapy and lifestyle changes.


Data Preprocessing and Preparation
Before using medical data in a deep learning model, it needs to be preprocessed to ensure the model can understand it. Preprocessing steps often include:

Text Tokenization: Converting textual data into sequences of numbers that the model can process.
Padding Sequences: Making all sequences the same length by adding padding characters at the beginning or end of shorter sequences.
Label Encoding: Converting categorical variables, such as disease names and medication names, into numerical labels.

Tokenizing and sequencing data

In [5]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(data['Patient_Problem'])

sequences = tokenizer.texts_to_sequences(data['Patient_Problem'])
#A 'tokenizer' variable is created to convert the textual data into sequences of integers. It only considers the top 5,000 words in the dataset in order to reduce the complexity. If the model encounters any out-of-vocabulary words during the training process then it will be replaced with the '<OOV>' token.

Padding Sequences
In order to make the input sequences have the same length, the code finds the longest sequence and pads all other sequences with zeros at the end ('post' padding) to match this sentence.

In [6]:
max_length = max(len(x) for x in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
#The 'pad_sequences' function is used to ensure that all the sequences have the same length. The 'maxlen' parameter is set to the maximum length of the sequences in the dataset. The 'padding' parameter is set to 'post' in order to add zeros at the end of the sequences.

Encoding the Labels and Converting them to Categorical
We will encode the 'Disease' and 'Prescription' columns as integers. Then the integer-encoded labels are converted into binary class matrices.

In [7]:
# Encoding the labels
label_encoder_disease = LabelEncoder()
label_encoder_prescription = LabelEncoder()

disease_labels = label_encoder_disease.fit_transform(data['Disease'])
prescription_labels = label_encoder_prescription.fit_transform(data['Prescription'])

# Converting labels to categorical
disease_labels_categorical = to_categorical(disease_labels)
prescription_labels_categorical = to_categorical(prescription_labels)

Combining Labels into a Multi-label Target Variable
Finally, now we will stack the binary class matrices together to form a single multi-label target variable 'Y'. This allows the model to predict both 'Disease' and 'Prescription' from the patient's problem.

In [8]:
Y = np.hstack((disease_labels_categorical, prescription_labels_categorical))

Model Building
Now, we will build the model using the LSTM and Sequential algorithm from TensorFlow. This model will learn from our preprocessed dataset to predict diseases based on patient symptoms.

Defining Model Architecture
We will use the 'Model' and 'Input' to define the model architecture, and 'Embedding' to convert the integer sequences into dense vectors of fixed size. We will use 'Dense' for output layers that make predictions.

In [9]:
input_layer = Input(shape=(max_length,))

embedding = Embedding(input_dim=5000, output_dim=64)(input_layer)
lstm_layer = LSTM(64)(embedding)

disease_output = Dense(len(label_encoder_disease.classes_), activation='softmax', 
name='disease_output')(lstm_layer)

prescription_output = Dense(len(label_encoder_prescription.classes_), 
activation='softmax', name='prescription_output')(lstm_layer)
#The model firstly have, an input layer that can handle sequences up to a certain length. Then there's an embedding layer that turns the numbers into vectors. After that, there's an LSTM layer that looks at the order of things, and finally, two dense layers that predict diseases and prescriptions using a softmax function for classification.

Compiling the model

In [10]:
model = Model(inputs=input_layer, outputs=[disease_output, prescription_output])

model.compile(
    loss={'disease_output': 'categorical_crossentropy', 
    'prescription_output': 'categorical_crossentropy'},
    optimizer='adam',
    metrics={'disease_output': ['accuracy'], 'prescription_output': ['accuracy']}
)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 17)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 17, 64)       320000      ['input_1[0][0]']                
                                                                                                  
 lstm (LSTM)                    (None, 64)           33024       ['embedding[0][0]']              
                                                                                                  
 disease_output (Dense)         (None, 178)          11570       ['lstm[0][0]']                   
                                                                                              

Training the model

In [11]:
model.fit(padded_sequences, {'disease_output': disease_labels_categorical, 'prescription_output':
      prescription_labels_categorical}, epochs=100, batch_size=32)

Epoch 1/100
13/13 [==============================] - 9s 28ms/step - loss: 11.1451 - disease_output_loss: 5.1792 - prescription_output_loss: 5.9660 - disease_output_accuracy: 0.0172 - prescription_output_accuracy: 0.0025
Epoch 2/100
13/13 [==============================] - 0s 16ms/step - loss: 11.1265 - disease_output_loss: 5.1634 - prescription_output_loss: 5.9631 - disease_output_accuracy: 0.0319 - prescription_output_accuracy: 0.0049
Epoch 3/100
13/13 [==============================] - 0s 15ms/step - loss: 11.0896 - disease_output_loss: 5.1247 - prescription_output_loss: 5.9649 - disease_output_accuracy: 0.0319 - prescription_output_accuracy: 0.0049
Epoch 4/100
13/13 [==============================] - 0s 15ms/step - loss: 10.9979 - disease_output_loss: 4.9951 - prescription_output_loss: 6.0028 - disease_output_accuracy: 0.0319 - prescription_output_accuracy: 0.0049
Epoch 5/100
13/13 [==============================] - 0s 14ms/step - loss: 10.8667 - disease_output_loss: 4.8988 - prescr

Making Predictions
The model is used to make predictions for new patients:

Pre-processed the patient's symptoms by performing tokenization and padding.
Feed the pre-processed data into the trained model.
The model predicts the disease and medication based on the patient's symptoms.
The predicted disease and medication will be presented.

In [14]:
def make_prediction(patient_problem):
    # Preprocessing the input
    sequence = tokenizer.texts_to_sequences([patient_problem])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')
    
    # Making prediction
    prediction = model.predict(padded_sequence)
    
    # Decoding the prediction
    disease_index = np.argmax(prediction[0], axis=1)[0]
    prescription_index = np.argmax(prediction[1], axis=1)[0]
    
    disease_predicted = label_encoder_disease.inverse_transform([disease_index])[0]
    prescription_predicted = label_encoder_prescription.inverse_transform([prescription_index])[0]
    
    print(f"Predicted Disease: {disease_predicted}")
    print(f"Suggested Prescription: {prescription_predicted}")


patient_input = "I've experienced a loss of appetite and don't enjoy food anymore."
make_prediction(patient_input)

1/1 [==============================] - 0s 52ms/step
Predicted Disease: Thyroid Nodule
Suggested Prescription: Pain management with medication; relaxation techniques.
